In [1]:
from LoadData import PartialDataSet
from Architecture.MSDC_CNNs import MSDC_CNNs
from Architecture.DenseNet import DenseNet
from Architecture.Inception import Inception
from Architecture.ResNet import resnet18
from CosineLoss import Cosine_Loss


from torchvision import datasets,transforms
from torch import nn,optim
from torch.utils.data import DataLoader,Subset
from torchsummary import summary
from tensorboardX import SummaryWriter
from tqdm.notebook import tqdm
from sklearn import manifold


import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import time

import os

In [2]:
def train_image_transform(crop_size,augmentation=True):
    if augmentation:
        return transforms.Compose([
            transforms.Grayscale(),
            transforms.ColorJitter(brightness=0.5,contrast=0.5,saturation=0.2,hue=0.1),
            transforms.RandomRotation(180),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(0.5),
            transforms.CenterCrop(crop_size),
            transforms.ToTensor(),
        ])
    else:
        return transforms.Compose([
        transforms.Grayscale(),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
    ])
        

def val_image_transform(crop_size):
    return transforms.Compose([
        transforms.Grayscale(),
        transforms.CenterCrop(crop_size),
        transforms.ToTensor(),
    ])

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
train_path = '../MSTAR_DataSet/17DEG_Train/'
val_path = '../MSTAR_DataSet/15DEG_Test/'

TARGET_SIZE = 84
TRAINING_RATIO = 0
SAMPLES = 15 # assert(TRAINING_RATIO==0 or SAMPLES==0)
FIX_SEGMENT = True
BATCHSIZE = 32
ARCHITECTURE = 'MSDC' # MSDC, DenseNet, ResNet, Inception
LOSSFUCTION = 'TwoStage' # TwoStage or Softmax
ALPHA = 0.6
TAU = 0.3
BETA = 0.05
EPOCH = 500
SEED = 168
DATA_AUGMENTATION = True

nowtime = time.strftime("%Y-%m-%d_%H", time.localtime())

if SAMPLES == 0:
    save_tag = '%s_%s_Ratio(%d)_%s_HyPara(a%.2f,b%.2f,t%.2f)'%(ARCHITECTURE,nowtime,TRAINING_RATIO,LOSSFUCTION,ALPHA,BETA,TAU)
else:
    save_tag = '%s_%s_Samples(%d)_%s_HyPara(a%.2f,b%.2f,t%.2f)'%(ARCHITECTURE,nowtime,SAMPLES,LOSSFUCTION,ALPHA,BETA,TAU)
    
saved_path = './SavedFiles/'+save_tag
if not os.path.exists(saved_path):
    os.mkdir(saved_path)
    os.mkdir(saved_path+'/TSNE_Sequence')

if ARCHITECTURE == 'MSDC':
    Classifier = MSDC_CNNs(growth_rate=12,block_config=(16,16,16),drop_rate=0.2,num_classes=10)
elif ARCHITECTURE == 'DenseNet':
    Classifier = DenseNet(growth_rate=12, block_config=(16, 16, 16), drop_rate=0.2, num_classes=10)
elif ARCHITECTURE == 'ResNet':
    Classifier = resnet18()
elif ARCHITECTURE == 'Inception':
    Classifier = DenseNet(growth_rate=12, block_config=(16, 16, 16), drop_rate=0.2, num_classes=10)
Classifier.cuda()

# summary(Classifier, (1, TARGET_SIZE, TARGET_SIZE))

cretierin = nn.CrossEntropyLoss()
cretierin.cuda()

if LOSSFUCTION == 'TwoStage':
    cosine_loss = Cosine_Loss(class_num=10,label_shift=TAU)
    cosine_loss.cuda()


optimizer = optim.SGD(Classifier.parameters(), lr=0.1, momentum=0.9, nesterov=True, weight_decay=0.0001,
                      dampening=0)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=150, gamma=0.3)

train_set = PartialDataSet(train_path,TRAINING_RATIO,sample=SAMPLES,transform=train_image_transform(TARGET_SIZE,DATA_AUGMENTATION),random_seed=SEED,fix_seg=FIX_SEGMENT)
test_set = datasets.ImageFolder(val_path,transform=val_image_transform(TARGET_SIZE))

train_loader = DataLoader(train_set,batch_size=BATCHSIZE,shuffle=True,num_workers=1)
val_loader = DataLoader(test_set,batch_size=BATCHSIZE,num_workers=1)

nowtime = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())

writer = SummaryWriter('./newlogs/'+save_tag)

data_size = {'train':len(train_loader),'val':len(val_loader)}

In [4]:
best_accuarcy = 0.0

for epoch in range(1,EPOCH+1):
    
    train_bar = tqdm(train_loader)

    running_loss = 0.0
    running_corrects = 0
    running_amount = 0

    for images,labels in train_bar:
        Classifier.train()

        images = images.cuda()
        labels = labels.cuda()

        outputs = Classifier(images)
        
        if LOSSFUCTION == 'TwoStage':
            if epoch < EPOCH*ALPHA:
                loss = cosine_loss(outputs,labels)
            else:
                loss = BETA*cretierin(outputs, labels) + cosine_loss(outputs,labels)
        else:
            loss = cretierin(outputs, labels)

        Classifier.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicts = torch.max(outputs, 1)

        running_loss += loss.item() * images.size(0)
        running_amount += images.size(0)
        batch_corrects = torch.sum(predicts == labels.data)
        running_corrects += batch_corrects

        train_bar.set_description(desc='Epoch:[%d/%d]'%(epoch,EPOCH))
        
    print('Epoch:[%d/%d], Corrects=%d/%d, Accuracy=%.2f%%, loss=%.4f'
        %(epoch,EPOCH,running_corrects,running_amount,float(running_corrects)/running_amount*100,loss.item()))
    
    epoch_loss = running_loss / data_size['train']
    epoch_acc = running_corrects.double() / running_amount

    writer.add_scalar('epoch/loss', epoch_loss, epoch)
    writer.add_scalar('epoch/Accuracy', epoch_acc, epoch)
    scheduler.step()

    if epoch % 1 == 0 or epoch == ALPHA*EPOCH-1:
        val_bar = tqdm(val_loader)
        running_loss = 0
        running_corrects = 0
        running_amount = 0
        val_bar.set_description(desc='Testing')
        
        predict_label = []
        true_label = []
        embedding_features = []
        with torch.no_grad():
            for images,labels in val_bar:
                Classifier.eval()
                images = images.cuda()
                labels = labels.cuda()

                outputs = Classifier(images)

                if LOSSFUCTION == 'TwoStage':
                    if epoch < int(ALPHA*EPOCH):
                        loss = cosine_loss(outputs,labels)
                    else:
                        loss = BETA*cretierin(outputs, labels) + cosine_loss(outputs,labels)
                else:
                    loss = cretierin(outputs, labels)

                _, predicts = torch.max(outputs, 1)

                running_loss += loss.item() * images.size(0)
                running_amount += images.size(0)
                batch_corrects = torch.sum(predicts == labels.data)
                running_corrects += batch_corrects

                embedding_features.extend(np.array(outputs.cpu()))
                predict_label.extend(np.array(predicts.cpu()))
                true_label.extend(np.array(labels.data.cpu()))
            
            print('Testing Results： Corrects = %d/%d, Accuracy = %.2f%%, loss = %.4f' % (
            running_corrects,running_amount,float(running_corrects)/running_amount*100, loss.item()))
            val_loss = running_loss / data_size['val']
            val_accuarcy = running_corrects.double() /running_amount

            writer.add_scalar('Val/loss', val_loss, epoch)
            writer.add_scalar('Val/Accuracy', val_accuarcy, epoch)


            '''t-SNE'''
            embedding_features = np.array(embedding_features)
            all_labels = np.array(true_label)
            n_samples, n_features = embedding_features.shape
            tsne = manifold.TSNE(n_components=2,init='pca')
            X_tsne = tsne.fit_transform(embedding_features)


            plt.figure(figsize=(6,6))
            x_min, x_max = X_tsne.min(0), X_tsne.max(0)
            X_norm = (X_tsne - x_min) / (x_max - x_min)
            for i in range(X_norm.shape[0]):
                plt.text(X_norm[i, 0], X_norm[i, 1], str(all_labels[i]), color=plt.cm.Set3(all_labels[i]), 
                         fontdict={'weight': 'bold', 'size': 9})
                
            plt.xlim(-0.1,1.1)
            plt.ylim(-0.1,1.1)
            plt.xticks([])
            plt.yticks([])
            plt.savefig(saved_path+'/TSNE_Sequence/%d_acc_%.4f.png'%(epoch,val_accuarcy),dpi=600)
            plt.close()

            if val_accuarcy > best_accuarcy:
                if os.path.exists(saved_path+'/%.4f.pkl'%best_accuarcy):
                    os.remove(saved_path+'/%.4f.pkl'%best_accuarcy)
                torch.save(Classifier, saved_path+'/%.4f.pkl'%val_accuarcy)
                
                df = pd.DataFrame({'Ground_Truth':true_label,'Predict_Labels':predict_label})
                if os.path.exists(saved_path+'/%.4f.csv'%best_accuarcy):
                    os.remove(saved_path+'/%.4f.csv'%best_accuarcy)
                df.to_csv(saved_path+'/%.4f.csv'%val_accuarcy)
                best_accuarcy = val_accuarcy
            
    if epoch == ALPHA*EPOCH-1:
        torch.save(Classifier, './model/' + 'CosineSwitchPoint.pkl')
        df = pd.DataFrame({'Ground_Truth':true_label,'Predict_Labels':predict_label})
        df.to_csv(saved_path+'/CosineSwitchPoint.csv')



Epoch:[1/500], Corrects=298/2747, Accuracy=10.85%, loss=0.0684



Testing Results： Corrects = 274/2425, Accuracy = 11.30%, loss = 0.0691



Epoch:[2/500], Corrects=336/2747, Accuracy=12.23%, loss=0.0668



Testing Results： Corrects = 275/2425, Accuracy = 11.34%, loss = 0.0657



Epoch:[3/500], Corrects=372/2747, Accuracy=13.54%, loss=0.0672


...


Testing Results： Corrects = 2249/2425, Accuracy = 92.74%, loss = 0.0009



Epoch:[499/500], Corrects=2737/2747, Accuracy=99.64%, loss=0.0019



Testing Results： Corrects = 2248/2425, Accuracy = 92.70%, loss = 0.0010



Epoch:[500/500], Corrects=2739/2747, Accuracy=99.71%, loss=0.0017



Testing Results： Corrects = 2246/2425, Accuracy = 92.62%, loss = 0.0009
